In [148]:
#Habilitar intellisense"
%config IPCompleter.greedy = True

- Completar los nombres de los integrantes del grupo
- Pueden agregar más celdas para elaborar las respuestas (crear funciones de apoyo, definir resultados intermedios, etc)
- Pueden apoyarse en búsquedas en internet y/o los notebooks para resolver los ejercicios.
- *La expresividad y claridad del código serán tomadas en cuenta*.
- Al finalizar, deberán enviar el notebook a jrocabado.t@ucb.edu.bo (**un** correo por grupo)
- **Cada 5 minutos de retraso en la entrega se penalizará con 2 puntos. Si se envían múltiples correos sólo se tomará en cuenta el último.** 

In [149]:
# Integrantes del grupo


**Las conversaciones con los clientes que realizaron pedidos a un restaurante están registradas en el dataframe ```pedidos```; el restaurante necesita extraer los siguientes datos de esas conversaciones en otro dataframe:**

- ```categoria```, valores posibles: ```venta```, ```consulta```
- ```importe_bs```, el importe (numérico o literal) en bolivianos cuando se haya realizado una venta. Si no se ha realizado una venta el importe es cero.
- ```item```, el item que se ha vendido (Ej. hamburguesas, sillpanchos, etc ). Si no se ha realizado una venta el item es ```ninguno```

In [150]:
import numpy as np
import pandas as pd
import spacy
from spacy import displacy
nlp = spacy.load("es_core_news_md")

In [151]:
pedidos = pd.read_csv("log_pedidos.csv")
pedidos

,cliente,restaurante
0,¿Todavía tiene pizza americana?,Sí
1,Queremos dos hamburguesas dobles,"Ok, son 70 Bs."
2,¿Todavía están recibiendo pedidos?,"Así es, atenderemos media hora más"
3,"Por favor, envíen dos sillpanchos especiales","Ok, le enviaremos la factura por 40 bolivianos"
4,"Por favor, envíen","Perdón, ¿enviar qué?"
5,Queremos un pique especial,"Son cien Bs., le enviaremos en 10 minutos"
6,Pique especial,"No entiendo, ¿quiere realizar un pedido?"


**Funciones de extracción**

In [152]:
from spacy.matcher import Matcher
#Add more lemmas as required
ORDER_VERBS_LEMMAS = ["querer", "enviar", "pedir", "comprar"]
nlp.get_pipe('attribute_ruler').add([[{"LOWER":"envíen"}]],{"LEMMA":"enviar"})
nlp.get_pipe('attribute_ruler').add([[{"TEXT":"Bs"}]],{"LEMMA":"boliviano"})

def has_question_mark(doc):
    return len([ t for t in doc if "PunctType=Qest" in t.morph]) > 0


def extract_item(doc):
    nouns = [t for t in doc if "NOUN" == t.pos_]
    nouns_with_order_verb_as_parent = [t for t in nouns if "VERB" == t.head.pos_ and t.head.lemma_ in ORDER_VERBS_LEMMAS ]
    return nouns_with_order_verb_as_parent[0] if len(nouns_with_order_verb_as_parent) > 0 else None
    
def extract_order_item(client_message):
    doc = nlp(client_message)
    return None if has_question_mark(doc) else extract_item(doc)


def extract_category(client_message):    
    order_item = extract_order_item(client_message)

    return "consulta" if order_item is None else "venta"
    
def extract_bs_amount(restaurant_answer):
    doc = nlp(restaurant_answer)
    matcher = Matcher(nlp.vocab)
    pattern = [{"LIKE_NUM":True},{"LEMMA":"boliviano"}]
    matcher.add("bs_amount_pattern", [pattern])
    matches = matcher(doc)
    for match_id, start, end in matches:
        return doc[start].text    
    
    
def extract_paid_amount(order_row):
    client_message = order_row["cliente"]
    restaurant_answer = order_row["restaurante"]
    category = extract_category(client_message)
    return 0 if "consulta" == category else extract_bs_amount(restaurant_answer)


def extract_ordered_item(order_row):
    client_message = order_row["cliente"]
    category = extract_category(client_message)
    return "ninguno" if "consulta" == category else extract_order_item(client_message)
    

**Extracción de datos**

In [153]:
category = pedidos["cliente"].apply(extract_category)
category

0    consulta
1       venta
2    consulta
3       venta
4    consulta
5       venta
6    consulta
Name: cliente, dtype: object

In [154]:
paid_amount = pedidos.apply(extract_paid_amount,axis='columns')
paid_amount

0       0
1      70
2       0
3      40
4       0
5    cien
6       0
dtype: object

In [155]:
ordered_item = pedidos.apply(extract_ordered_item,axis='columns')
ordered_item

0         ninguno
1    hamburguesas
2         ninguno
3     sillpanchos
4         ninguno
5           pique
6         ninguno
dtype: object

In [156]:
extracted_order_data = pd.DataFrame({"categoria":category, "importe_bs":paid_amount, "item": ordered_item})
extracted_order_data

,categoria,importe_bs,item
0,consulta,0,ninguno
1,venta,70,hamburguesas
2,consulta,0,ninguno
3,venta,40,sillpanchos
4,consulta,0,ninguno
5,venta,cien,pique
6,consulta,0,ninguno
